# 04 - Feature Store with Feast

This notebook demonstrates the Feast feature store setup for identity risk scoring.

## Key Benefits
- **Online/Offline Consistency**: Same features for training and serving
- **Feature Versioning**: Track feature definitions over time
- **Low-Latency Serving**: SQLite online store for real-time scoring
- **Multi-Tenant Ready**: Feature isolation per tenant

In [ ]:
import sys
sys.path.insert(0, '..')

from feast import FeatureStore
import pandas as pd
from datetime import datetime

## Connect to Feature Store

In [ ]:
store = FeatureStore(repo_path='../feature_store')

print("Feature Views:")
for fv in store.list_feature_views():
    print(f"  - {fv.name}: {[f.name for f in fv.features]}")

print("\nFeature Services:")
for fs in store.list_feature_services():
    print(f"  - {fs.name}")

## Online Feature Retrieval

Used for real-time scoring - low latency (<10ms)

In [ ]:
# Get features for specific users
user_ids = ['user_0001', 'user_0010', 'user_0100']

features = store.get_online_features(
    features=[
        'user_login_features:ip_reputation_score',
        'user_login_features:is_new_device',
        'user_login_features:device_age_days',
        'user_login_features:mfa_used',
    ],
    entity_rows=[{'user_id': uid} for uid in user_ids]
).to_df()

print("Online Features:")
features

## Historical Feature Retrieval

Used for training - point-in-time correct features

In [ ]:
# Create entity DataFrame with timestamps
entity_df = pd.DataFrame({
    'user_id': ['user_0001', 'user_0002', 'user_0003'],
    'event_timestamp': pd.to_datetime(['2026-01-15', '2026-01-16', '2026-01-17'])
})

# Get historical features
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        'user_login_features:ip_reputation_score',
        'user_login_features:is_new_device',
        'user_login_features:mfa_used',
    ]
).to_df()

print("Historical Features:")
training_df

## Using Feature Service

Feature services bundle related features for specific use cases

In [ ]:
# Get feature service
risk_service = store.get_feature_service('risk_scoring')
print(f"Service: {risk_service.name}")
print(f"Features: {[f.name for fv in risk_service.feature_view_projections for f in fv.features]}")

## Materialization

Push features from offline to online store

In [ ]:
# Materialize features (run in CLI: feast materialize-incremental <end_date>)
from datetime import datetime, timedelta

print("To materialize features, run:")
print(f"  cd ../feature_store && feast materialize-incremental {datetime.now().isoformat()}")